In [155]:
import pandas as pd
import numpy as np
import joblib
pd.options.mode.chained_assignment = None

test_a_base = pd.read_csv('data/test_a_base.csv')
test_a_op = pd.read_csv('data/test_a_op.csv')
test_a_trans = pd.read_csv('data/test_a_trans.csv')

In [223]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', 100)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

In [213]:
ohe_encoder = joblib.load('ohe_encoder.pkl')

In [214]:
test_a_base_non_nan = test_a_base.drop(columns=['service3_level'],axis=1)

test_a_base_non_nan['sex'].fillna('category 0',inplace=True)
test_a_base_non_nan['provider'].fillna('category 0',inplace=True)
test_a_base_non_nan['province'].fillna('21c43413032d5522',inplace=True)
test_a_base_non_nan['city'].fillna('e3685345c54a1cef',inplace=True)

user = test_a_base_non_nan['user'].values

category_columns_name = ['sex','provider','level','verified','regist_type','agreement1','agreement2','agreement3','agreement4']
test_a_base_non_user = test_a_base_non_nan.drop(columns=['user'],axis=1)
one_hot_value = ohe_encoder.transform(test_a_base_non_user[category_columns_name]).toarray()
one_hot_df = pd.DataFrame(one_hot_value)

In [215]:
test_a_base_non_user.head()

,sex,age,provider,level,verified,using_time,regist_type,card_a_cnt,card_b_cnt,card_c_cnt,...,balance2_avg,service3,product1_amount,product2_amount,product3_amount,product4_amount,product5_amount,product6_amount,product7_cnt,product7_fail_cnt
0,category 0,24853,category 0,category 1,category 0,24713,category 1,24719,24712,24712,...,level 5,category 0,level 1,level 1,level 1,level 0,level 0,level 1,24712,24706
1,category 1,25011,category 0,category 1,category 0,24743,category 7,24712,24712,24706,...,level 1,category 0,level 1,level 1,level 1,level 0,level 0,level 1,24706,24706
2,category 1,24877,category 0,category 2,category 0,24744,category 7,24719,24725,24712,...,level 5,category 0,level 2,level 19,level 1,level 0,level 0,level 19,24712,24706
3,category 0,24925,category 0,category 2,category 0,24715,category 1,24712,24712,24706,...,level 5,category 0,level 1,level 1,level 1,level 0,level 0,level 1,24712,24706
4,category 1,24877,category 2,category 1,category 0,24706,category 3,24712,24712,24706,...,level 1,category 0,level 1,level 1,level 1,level 0,level 0,level 1,24712,24706


In [216]:
test_a_base_categoty = test_a_base_non_user.select_dtypes('O')

for c in list(test_a_base_categoty.columns[0:9])+['service3']:
    test_a_base_categoty[c]=test_a_base_categoty[c].apply(lambda x:int(x.strip().split(' ')[1]))
    
for c in list(test_a_base_categoty.columns[11:17])+list(test_a_base_categoty.columns[18:25]):
    test_a_base_categoty[c]=test_a_base_categoty[c].apply(lambda x:int(x.strip().split(' ')[1]))

In [217]:
city_dic=joblib.load('city_dic.pkl')
province_dic=joblib.load('province_dic.pkl')

In [218]:
test_a_base_categoty['city'] = test_a_base_categoty['city'].apply(lambda x:city_dic[x])
test_a_base_categoty['province'] = test_a_base_categoty['province'].apply(lambda x:province_dic[x])

In [219]:
test_a_base_categoty.drop(columns=category_columns_name,axis=1,inplace=True)

In [220]:
number_value = test_a_base_non_user.select_dtypes('int64').values
df_value= pd.DataFrame(number_value)
df_value.columns = test_a_base_non_user.select_dtypes('int64').columns

df_value['product7_success_cnt'] = df_value['product7_cnt']-df_value['product7_fail_cnt']
df_value['card_cnt'] = df_value['card_a_cnt']+df_value['card_b_cnt']+df_value['card_c_cnt']+df_value['card_d_cnt']
df_value['ip_cnt_avg'] = df_value['ip_cnt']/df_value['login_days_cnt']
df_value['login_cnt_period']=df_value['login_cnt_period1']+df_value['login_cnt_period2']
df_value['login_cnt_period1_avg']=df_value['login_cnt_period1']/df_value['login_days_cnt']
df_value['login_cnt_period2_avg']=df_value['login_cnt_period2']/df_value['login_days_cnt']
df_value['login_cnt_period_avg']=df_value['login_cnt_period']/df_value['login_days_cnt']
df_value['using_time_avg'] = df_value['using_time']/df_value['login_days_cnt']
df_value['product7_success_cnt_avg'] = df_value['product7_success_cnt']/df_value['login_days_cnt']
df_value['product7_cnt_avg']=df_value['product7_cnt']/df_value['login_days_cnt']
df_value['product7_fail_cnt_avg']=df_value['product7_fail_cnt']/df_value['login_days_cnt']
df_value['service_cnt']=df_value['service1_cnt']+df_value['service2_cnt']
df_value['service_cnt_avg']= df_value['service_cnt']/df_value['login_days_cnt']
df_value['service_amt_cnt_avg']=df_value['service1_amt']/df_value['service_cnt']

In [221]:
df_value = (df_value-df_value.min())/(df_value.max()-df_value.min())
df_base = pd.DataFrame()
df_base['user'] = user

df_base = pd.concat([df_base, test_a_base_categoty,df_value,one_hot_df],axis=1)

In [222]:
df_base.shape

(24315, 76)

In [224]:
df_base.head()

,user,province,city,balance,balance_avg,balance1,balance1_avg,balance2,balance2_avg,service3,product1_amount,product2_amount,product3_amount,product4_amount,product5_amount,product6_amount,age,using_time,card_a_cnt,card_b_cnt,card_c_cnt,op1_cnt,op2_cnt,card_d_cnt,agreement_total,service1_cnt,service1_amt,service2_cnt,acc_count,login_cnt_period1,login_cnt_period2,ip_cnt,login_cnt_avg,login_days_cnt,product7_cnt,product7_fail_cnt,product7_success_cnt,card_cnt,ip_cnt_avg,login_cnt_period,login_cnt_period1_avg,login_cnt_period2_avg,login_cnt_period_avg,using_time_avg,product7_success_cnt_avg,product7_cnt_avg,product7_fail_cnt_avg,service_cnt,service_cnt_avg,service_amt_cnt_avg,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,TestA_21427,0.227933,0.242705,16,14,1,1,18,5,0,1,1,1,0,0,1,0.344262,0.132075,0.101562,0.088235,0.065217,0.073446,0.101562,0.000000,0.422414,0.0,0.0,0.0,0.021834,0.014139,0.010796,0.049462,0.026971,0.178832,0.032787,0.0,0.857143,0.102459,0.241382,0.014369,0.013135,0.007052,0.012579,0.764201,0.853756,0.649362,0.643059,0.0,0.808871,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1,TestA_08694,0.357078,0.318182,1,1,1,1,1,1,0,1,1,1,0,0,1,0.714286,0.698113,0.046875,0.088235,0.000000,0.033898,0.000000,0.000000,0.112069,0.0,0.0,0.0,0.016594,0.001073,0.003243,0.006989,0.076763,0.023723,0.000000,0.0,0.000000,0.049180,0.244883,0.001741,0.000922,0.002831,0.001504,0.963471,0.000000,0.765509,0.767155,0.0,0.964964,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,TestA_13449,0.212571,0.242321,20,20,20,19,20,5,0,2,19,1,0,0,19,0.400468,0.716981,0.101562,0.279412,0.065217,0.209040,0.000000,0.315789,0.637931,0.0,0.0,0.0,0.048035,0.102073,0.113136,0.141398,0.126556,0.479927,0.032787,0.0,0.857143,0.180328,0.244883,0.113175,0.100345,0.105847,0.109899,0.534750,0.848115,0.409139,0.404580,0.0,0.508900,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
3,TestA_10937,0.221616,0.176101,12,12,1,1,13,5,0,1,1,1,0,0,1,0.512881,0.169811,0.046875,0.088235,0.000000,0.175141,0.046875,0.000000,0.267241,0.0,0.0,0.0,0.016594,0.041128,0.087069,0.134409,0.114108,0.399635,0.032787,0.0,0.857143,0.049180,0.262889,0.056736,0.038936,0.080897,0.053222,0.559913,0.849612,0.472887,0.467865,0.0,0.588504,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,TestA_14194,0.227933,0.168000,1,1,1,1,1,1,0,1,1,1,0,0,1,0.400468,0.000000,0.046875,0.088235,0.000000,0.000000,0.000000,0.000000,0.163793,0.0,0.0,0.0,0.016594,0.000340,0.001026,0.003226,0.051867,0.010949,0.032787,0.0,0.857143,0.049180,0.244883,0.000551,0.000266,0.000815,0.000433,0.911344,0.856935,0.784698,0.777413,0.0,0.977867,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


### OP

In [203]:
op_df = pd.DataFrame()
group = test_a_op.groupby(['user']).count()
op_df['user'] = group.index

count_column=test_a_op.columns[1:-2]
for column in count_column:
    print(column)
    op_df['op_'+column+'_count'] = test_a_op.groupby('user')[column].count().values
    op_df['op_'+column+'_count'] = test_a_op.groupby('user')[column].nunique().values
#     des_df = test_a_op.groupby('user')[column].describe()
#     op_df['op_'+column+'_count'] = des_df['count'].values
#     op_df['op_'+column+'_n'] = des_df['unique'].values
#     op_df['op_'+column+'_fre'] = des_df['freq'].values

op_type
op_mode
op_device
ip
net_type
channel
ip_3


In [204]:
op_df_tmp = op_df

In [205]:
from datetime import timedelta
def timedelta2sec(delta):
    day = delta.split('days')[0].strip()
    h,m,s = delta.split('days')[1].strip().split(':')
    sec=timedelta(days=int(day),hours=int(h),minutes=int(m),seconds=float(s)).total_seconds()
    return sec

test_a_op['time_diff_sec'] = test_a_op['tm_diff'].apply(timedelta2sec)

for operate in ['max','min','mean','median','std']:
    op_df['op_time_'+operate]=test_a_op.groupby('user')['time_diff_sec'].agg(operate).values
    
op_df.fillna(0,inplace=True)

In [206]:
op_df.shape

(20458, 13)

### trans

In [189]:
trans_df = pd.DataFrame()
group = test_a_trans.groupby(['user']).count()
trans_df['user'] = group.index

count_column = list(test_a_trans.columns[1:4])+list(test_a_trans.columns[5:9]) 
for column in count_column:
    print(column)
    trans_df['trans_'+column+'_count'] = test_a_trans.groupby('user')[column].count().values
    trans_df['trans_'+column+'_count'] = test_a_trans.groupby('user')[column].nunique().values
#     des_df = test_a_trans.groupby('user')[column].describe()
#     trans_df['trans_'+column+'_count'] = des_df['count'].values
#     trans_df['trans_'+column+'_n'] = des_df['unique'].values
#     trans_df['trans_'+column+'_fre'] = des_df['freq'].values

platform
tunnel_in
tunnel_out
type1
ip
type2
ip_3


In [190]:
trans_df_tmp = trans_df

In [191]:
trans_df_tmp.shape

(18813, 8)

In [192]:
trans_df = trans_df_tmp

In [193]:
def timedelta2sec(delta):
    day = delta.split('days')[0].strip()
    h,m,s = delta.split('days')[1].strip().split(':')
    sec=timedelta(days=int(day),hours=int(h),minutes=int(m),seconds=float(s)).total_seconds()
    return sec

In [194]:
for operate in ['max','min','mean','median','std']:
    trans_df['trans_amount_'+operate]=test_a_trans.groupby(['user'])['amount'].agg(operate).values
    
test_a_trans['time_diff_sec'] = test_a_trans['tm_diff'].apply(timedelta2sec)

for operate in ['max','min','mean','median','std']:
    trans_df['trans_time_'+operate]=test_a_trans.groupby('user')['time_diff_sec'].agg(operate).values

trans_df.fillna(0,inplace=True)

In [201]:
trans_df.shape

(18813, 18)

## 数据合并

In [196]:
df_base.shape

(24315, 76)

In [227]:
test_df = pd.merge(df_base, op_df, on='user', how='left')


In [229]:
test_df = pd.merge(test_df, trans_df, on='user', how='left')

In [230]:
test_df.shape

(24315, 105)

In [209]:
model = joblib.load('model.pkl')

In [210]:
x = test_df.drop('user',axis=1)
res = model.predict_proba(x)[:,1]

ValueError: Number of features of the model must match the input. Model n_features_ is 103 and input n_features is 104 

In [146]:
df_res = pd.DataFrame()
df_res['user'] = test_a_base['user']
df_res['prob'] = res

In [151]:
df_res.to_csv('result4.csv', index=False)

In [111]:
len(test_a_base)

24315

In [142]:
np.mean(res)

0.2631698898095935

In [150]:
np.mean(res)

0.260304385163033